**Inputs**
* ImageCollection with Images: maximum a posterior (MAP) probabilities of each magnitude-year pair

**Outputs**
* ImageCollection with Images: maximum a posterior (MAP) probabilities of each magnitude-year pair--where magnitude values are thresholds

In [1]:
import ee
ee.Authenticate()

Enter verification code: 4/1AX4XfWjvXR_YFR7XNU0UABaiHjxqFKUlEXNQpmJ5zbfQHmemc58hzTIq1fE

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
WHOLE_GLOBE = ee.Geometry.Rectangle([-179.999, -90, 180, 90], 'EPSG:4326', False)
START_YEAR = 2020
END_YEAR = 2099
use_greaterthan = True

In [4]:
hazard_info = {
    'ds': {
        'num_bins': 100,
        'bin_width': 1
    },
    'maxdryspell': {
        'num_bins': 73,
        'bin_width': 5
    }
}

In [15]:
for hazardname in hazard_info.keys():
    mapprobs = ee.ImageCollection('users/tedwongwri/dataportal/MAPprobs/{0}'.format(hazardname))
    for year in range(START_YEAR, END_YEAR + 1):
        year_list = ee.List([])
        img = mapprobs.filterMetadata('year', 'equals', year).first()
        band_names = img.bandNames().getInfo()
        mag_parts = [i.split('_')[1] for i in band_names]
        highlows = [[float(j) for j in i.split('-')] for i in mag_parts]
        for magnitude in highlows:
            total_img = ee.Image(0)
            prob_img = ee.Image(0)
            for hl1 in highlows:
                prob_segment = img.select('magnitude_{0}-{1}'.format(str(int(hl1[0])), str(int(hl1[1]))))
                total_img = total_img.add(prob_segment)
                prob_img = prob_img.add(prob_segment.multiply(((hl1[0] >= magnitude[1] and use_greaterthan) or (hl1[1] <= magnitude[0] and not use_greaterthan)) * 1))
            year_list = year_list.add(prob_img.divide(total_img).set('threshold', [magnitude[1], magnitude[0]][use_greaterthan * 1]))
        year_img = ee.ImageCollection(year_list).toBands().rename(['threshold_{0}'.format(str(int(i[[1, 0][use_greaterthan * 1]]))) for i in highlows]).set('year', year)
        task = ee.batch.Export.image.toAsset(**{
            'image': year_img,
            'description': 'thresholdMAPprobs_{0}_{1}'.format(hazardname, str(year)),
            'assetId': 'users/tedwongwri/dataportal/thresholdMAPprobs/{0}/{1}'.format(hazardname, str(year)),
            'region': WHOLE_GLOBE,
            'crs': 'EPSG:4326',
            'dimensions': '1440x720'
        })
        task.start()